<a href="https://colab.research.google.com/github/E-CG/AI4ENG/blob/master/02%20-%20Preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **📦 Librerias y paquetes para la ejecución del notebook.**

In [ ]:
! pip install py7zr

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install kaggle

In [4]:
! mkdir ~/.kaggle
# Aquí debes cambiar la dirección donde tengas tus credenciales de Kaggle
! cp /content/drive/MyDrive/Modelos_I/credentials_kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download favorita-grocery-sales-forecasting
! unzip favorita-grocery-sales-forecasting.zip

In [1]:
# Librerias uso básico
import numpy as np
import pandas as pd
import math as m
import time
import py7zr
import os
from subprocess import check_output

# Librerias preprocesado
from mlxtend.preprocessing import minmax_scaling

# Librerias para gráficar
import seaborn as sns
import matplotlib.pyplot as plt

# Funciones de sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression,SGDRegressor,ElasticNet,Ridge
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error

# **🗂️Leyendo y extrayendo los archivos .csv**

In [7]:
# Ruta al archivo 7z en Google Drive
sevenzip_file_path = '/content'

# Directorio de destino para la extracción
extracted_dir = '/content/extracted_data/'

# Crear el directorio de destino si no existe
os.makedirs(extracted_dir, exist_ok=True)

# Listar los archivos .7z en el directorio de entrada
files_to_extract = [file for file in os.listdir(sevenzip_file_path) if file.endswith('.7z')]

# Iterar a través de los archivos y descomprimirlos
for file_to_extract in files_to_extract:
    with py7zr.SevenZipFile(os.path.join(sevenzip_file_path, file_to_extract), mode='r') as z:
        z.extractall(path=extracted_dir)

In [2]:
stores = pd.read_csv('/content/extracted_data/stores.csv')
items = pd.read_csv('/content/extracted_data/items.csv')
holidays_e = pd.read_csv('/content/extracted_data/holidays_events.csv', parse_dates=["date"])
transactions = pd.read_csv('/content/extracted_data/transactions.csv', parse_dates=["date"])
oil = pd.read_csv('/content/extracted_data/oil.csv')

# Cargar el archivo de entrenamiento en chunks
chunked_dfs = pd.read_csv("/content/extracted_data/train.csv",
                          chunksize=20000,
                          usecols=[1, 2, 3, 4, 5],
                          parse_dates=['date'],
                          low_memory=False)

print('Archivos cargados 🗞️✅')

Archivos cargados 🗞️✅


# **❌¿Qué archivos tienen datos nulos?**

🖊️ Se toma cada columna del dataframe en cuestion, se suman la cantidad de registros NaN y se convierte en un **porcentaje**. A cada columna se le saca tal porcentaje.

In [ ]:
oil_nan = (oil.isnull().sum() / oil.shape[0]) * 100
oil_nan

date          0.000000
dcoilwtico    3.530378
dtype: float64

Hay un 3.5% de datos faltantes en el archivo (oil.csv)

In [ ]:
store_nan = (stores.isnull().sum() / stores.shape[0]) * 100
store_nan

store_nbr    0.0
city         0.0
state        0.0
type         0.0
cluster      0.0
dtype: float64

No hay datos faltantes en (stores.csv)

In [ ]:
item_nan = (items.isnull().sum() / items.shape[0]) * 100
item_nan

item_nbr      0.0
family        0.0
class         0.0
perishable    0.0
dtype: float64

No hay datos faltantes en (items.csv)

In [ ]:
holiday_nan = (holidays_e.isnull().sum() / holidays_e.shape[0]) * 100
holiday_nan

date           0.0
type           0.0
locale         0.0
locale_name    0.0
description    0.0
transferred    0.0
dtype: float64

No hay datos nulo en el archivo (holidays_events.csv)

In [ ]:
tran_nan = (transactions.isnull().sum() / transactions.shape[0]) * 100
tran_nan

date            0.0
store_nbr       0.0
transactions    0.0
dtype: float64

No hay datos faltantes en (transactions.csv)

# **🦾Primeras predicciones**

Tomar 1 tienda por estado en las fechas mayores a 2016 y predecir las ventas unitarias de todos los productos.

In [3]:
# Agrupar por estado y obtener el primer store_nbr en cada estado
tienda_por_estado = stores.groupby('state')['store_nbr'].first()
tienda_por_estado

state
Azuay                             37
Bolivar                           19
Chimborazo                        14
Cotopaxi                          12
El Oro                            40
Esmeraldas                        43
Guayas                            24
Imbabura                          15
Loja                              38
Los Rios                          31
Manabi                            52
Pastaza                           22
Pichincha                          1
Santa Elena                       25
Santo Domingo de los Tsachilas     5
Tungurahua                        23
Name: store_nbr, dtype: int64

In [4]:
# Inicializar una lista para almacenar los DataFrames filtrados
datos_filtrados = []

# Iterar a través de los chunks
for chunk in chunked_dfs:
    # Crear una máscara booleana para las tiendas seleccionadas
    mask_tiendas = chunk['store_nbr'].isin(stores['store_nbr'])
    # Crear una máscara booleana para las fechas entre 2016-01-01 a 2016-12-31
    mask = (chunk['date'] >= '2016-01-01') & (chunk['date'] <= '2016-12-31') & chunk['store_nbr'].isin(tienda_por_estado)

    # Aplicar la máscara y agregar las filas filtradas a la lista
    trozo_filtrado = chunk[mask]
    datos_filtrados.append(trozo_filtrado)

# Concatenar los DataFrames filtrados en uno solo
train_store_state = pd.concat(datos_filtrados, ignore_index=True)

In [ ]:
train_nan = (train_store_state.isnull().sum() / train_store_state.shape[0]) * 100
train_nan

date           0.0
store_nbr      0.0
item_nbr       0.0
unit_sales     0.0
onpromotion    0.0
dtype: float64

No hay datos nulos en el dataframe de entrenamiento

## **💱Convirtiendo datos para que la máquina lo pueda entender.**

In [5]:
# Convirtiendo las fechas
train_store_state['date'] = pd.to_datetime(train_store_state['date'])
holidays_e['date'] = pd.to_datetime(holidays_e['date'])
oil['date'] = pd.to_datetime(oil['date'])

In [6]:
# Columnas que no son importantes para las predicciones
# No presentan ningun efecto, ni relación (Veáse la exploración de datos.)
stores.drop(columns=['state', 'type', 'cluster'], inplace= True)

''' Si el día no es festivo se considera normal.
Si no es festivo nacional o local, también se considera normal. '''
holidays_e.drop(columns = ['locale_name', 'description', 'transferred'], inplace = True)

## **📄Juntando los demás archivos en train.csv y generando dataframes de prueba y entrenamiento**

In [7]:
class MergeDataTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Constructor de la clase
        print("Inicializando MergeDataTransformer")

    def fit(self, X, y=None):
        # Este método no realiza ningún ajuste de datos, por lo que simplemente devuelve 'self'
        return self

    def transform(self, X):
        # Realiza la fusión de DataFrames
        X = pd.merge(pd.merge(pd.merge(X, stores, on='store_nbr', how='left'),
                             oil, on='date', how='left'),
                             holidays_e, on='date', how='left')

        # Ordena los datos por fecha
        X = X.sort_values('date')

        # Reemplazando NaNs en oil.csv
        X.dcoilwtico = X.dcoilwtico.fillna(method='bfill', axis=0).fillna(0)

        # Llena los valores nulos en 'type' y 'locale' con 'Normal'
        X.type = X.type.fillna('Normal')
        X.locale = X.locale.fillna('Normal')

        # Aplica la codificación one-hot a las columnas categóricas especificadas
        encoder = OneHotEncoder()
        categorical_cols = ['store_nbr', 'item_nbr', 'onpromotion', 'city', 'type', 'locale']

        encoder.fit(X[categorical_cols])
        encoded_cols = encoder.transform(X[categorical_cols]).toarray()
        encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))

        # Concatena los datos codificados one-hot con el DataFrame original
        X = pd.concat([X.drop(columns=categorical_cols), encoded_df], axis=1)

        # Restablece el índice del DataFrame resultante y elimina la columna 'index'
        data = X.reset_index().drop(columns='index')

        # Devuelve el DataFrame resultante
        return data

In [8]:
class SplitDataTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Constructor de la clase
        print("Inicializando SplitDataTransformer")

    def fit(self, X, y=None):
        # Este método no realiza ningún ajuste de datos, por lo que simplemente devuelve 'self'
        return self

    def transform(self, X):
        # Extrae la columna 'unit_sales' como el objetivo 'y' y elimina esta columna de 'X'
        y = X.pop('unit_sales')

        # Divide los datos en conjuntos de entrenamiento y prueba (90% de entrenamiento, 10% de prueba)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

        # Establece la columna 'date' como el índice en los conjuntos de entrenamiento y prueba
        X_train = X_train.set_index('date')
        X_test = X_test.set_index('date')

        # Devuelve los conjuntos de entrenamiento y prueba, así como los objetivos de entrenamiento y prueba
        return X_train, X_test, y_train, y_test

In [9]:
pipeline = Pipeline([
    ('merge_data', MergeDataTransformer()),
    ('split_data', SplitDataTransformer())
])

Inicializando MergeDataTransformer
Inicializando SplitDataTransformer


In [10]:
# Seleccionar aleatoriamente 80,000 filas de 'train_store_state'
train_sample = train_store_state.sample(n=78000)

# Obtener los identificadores únicos de los productos en el conjunto de muestra
productos_sample = train_sample['item_nbr'].unique()

# Tomar los primeros 3 productos únicos
productos_sample = productos_sample[:3]

In [11]:
X_train, X_test, y_train, y_test = pipeline.fit_transform(train_sample)

Viendo los conjuntos de prueba y entrenamiento

In [ ]:
X_train.head(2)

In [ ]:
X_test.head(2)

In [ ]:
y_train.head(2)

In [ ]:
y_test.head(2)

## **🔎Utilizando algunos modelos para hacer las primeras predicciones**

In [12]:
# Definiendo una función para evaluar el rendimiento de cada modelo a utilizar
def checkModelPerformane(model):
  # Entrenar el modelo con los datos de entrenamiento
  model.fit(X_train.values, y_train.values)

  # Realizar predicciones en el conjunto de prueba
  predictions = model.predict(X_test.values)

  # Calcular y mostrar el error cuadrático medio (RMSE) de las predicciones
  rmse = np.sqrt(mean_squared_error(y_test.values, predictions))
  print("Root Mean Squared Error (RMSE): ", rmse)

  # Calcular y mostrar el error absoluto medio (MAE) de las predicciones
  mae = np.sqrt(mean_absolute_error(y_test.values, predictions))
  print("Root Mean Absolute Error (MAE): ", mae)

Menores valores en el estadistico de error, mejor será el modelo.

In [13]:
print("LinearRegression")
checkModelPerformane(LinearRegression())

LinearRegression
Root Mean Squared Error (RMSE):  9489713153.762894
Root Mean Absolute Error (MAE):  13224.865445601821


In [15]:
print("SGDRegressor")
checkModelPerformane(SGDRegressor())

SGDRegressor
Root Mean Squared Error (RMSE):  48486333.30887891
Root Mean Absolute Error (MAE):  4705.666659393304


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [16]:
print("ElasticNet")
checkModelPerformane(ElasticNet())

ElasticNet
Root Mean Squared Error (RMSE):  10.594168981160163
Root Mean Absolute Error (MAE):  2.296691744213001


In [14]:
print("Ridge")
checkModelPerformane(Ridge())

Ridge
Root Mean Squared Error (RMSE):  11.363234521196532
Root Mean Absolute Error (MAE):  2.3523794464919305
